# Time Series Classification Algorithms - (9) Image Based

<div style="border: 1px solid #007acc; background-color: #e6f4ff; padding: 10px; border-radius: 5px; color: black;">
  <strong>📘 Info:</strong> I do not claim ownership of the content in this notebook. It is based on official <a href="https://www.aeon-toolkit.org/en/latest/examples.html" target="_blank">aeon tutorials</a>, and parts of the code or text may have been copy-pasted or adapted directly from those or other sources for learning purposes.
</div>

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
import matplotlib.pyplot as plt

import numpy as np

import pandas as pd

import seaborn as sns

from sklearn import metrics

import time

In [ ]:
from aeon.datasets import load_classification

In [ ]:
DATASET_NAMES = [
    "MelbournePedestrian",
    "ArrowHead",
    "Colposcopy"
]

X_train_dict, y_train_dict = {}, {}
X_test_dict, y_test_dict = {}, {}

accuracy_dict, f1_score_dict, duration_dict = {}, {}, {}

for dataset_name in DATASET_NAMES: 
    X_train_dict[dataset_name], y_train_dict[dataset_name] = load_classification(
        dataset_name, split="train"
    )
    X_test_dict[dataset_name], y_test_dict[dataset_name] = load_classification(
        dataset_name, split="test"
    )

    X_train_dict[dataset_name] = X_train_dict[dataset_name].squeeze(1)
    X_test_dict[dataset_name] = X_test_dict[dataset_name].squeeze(1)

    accuracy_dict[dataset_name] = {}
    f1_score_dict[dataset_name] = {}
    duration_dict[dataset_name] = {}

```bibtex
@inbook{faouzi2024,
  author = {Johann Faouzi},
  title = {Time Series Classification: A Review of Algorithms and Implementations},
  year = {2024},
  month = {March},
  booktitle = {Advances in Time Series Analysis and Forecasting},
  publisher = {IntechOpen},
  isbn = {978-0-85466-053-7},
  doi = {10.5772/intechopen.1004810},
}
```

"In order to investigate temporal correlations between all the pairs of observations, several methods relying on transforming time series (i.e., vectors) into images (i.e., matrices) have been proposed." \cite{faouzi2024}

## 1. Recurrence Plot

```bibtex
@article{eckmann1987,
  author = {J.-P. Eckmann and S. O. Kamphorst and D. Ruelle},
  title = {Recurrence plots of dynamical systems},
  journal = {EPL (Europhysics Letters)},
  volume = {4},
  number = {9},
  pages = {973--977},
  year = {1987},
  publisher = {IOP Publishing}
}
```

Proposed by \cite{eckmann1987}.

"A recurrence plot is an old technique that was originally introduced to visually inspect time constancy in dynamical systems through trajectories. A trajectory is defined as a subsequence of equally spaced values." \cite{faouzi2024}

$$
\vec{x}_i = \left( x_i, x_{i + \tau}, \ldots, x_{i + (m - 1)\tau} \right), \quad \forall i \in \{1, \ldots, n - (m - 1)\tau \}
$$

"where $m$ is the length of the trajectory and $\tau$ is the time delay, that is, the time gap between back-to-back time points in the trajectory." \cite{faouzi2024}

"A recurrence plot, denoted by $R$, is a matrix consisting of the binarized pairwise distances between all the pairs of trajectories from a time series:" \cite{faouzi2024}

$$
\forall i, j \in \{1, \ldots, n - (m - 1)\tau \}, \quad 
R_{ij} = \Theta\left( \varepsilon - \left\| \vec{x}_i - \vec{x}_j \right\|_2 \right)
$$

"where $\varepsilon$ is the threshold used to binarize the distance and Θ is the Heaviside step function, which is 1 for positive arguments and 0 for non-positive arguments. Visually, as a black-and-white image, a pixel is black if and only if the distance between the two considered trajectories is smaller than the threshold." \cite{faouzi2024}



## 2. Gramian Angular Field

## 3. Markov Transition Field

## 4. Comparison between the Classifiers